In [109]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [111]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [113]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [115]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [119]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d']])

In [121]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled_data, columns=['price_change_percentage_24h', 'price_change_percentage_7d'])

# Copy the crypto names from the original DataFrame
df_scaled['coin_id'] = df_market_data.index

# Set the coin_id column as index
df_scaled.set_index('coin_id', inplace=True)

# Display the scaled DataFrame
df_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d
coin_id,,
bitcoin,0.508529,0.493193
ethereum,0.185446,0.934445
tether,0.021774,-0.706337
ripple,-0.040764,-0.810928
bitcoin-cash,1.193036,2.000959


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [125]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [127]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # 1. Create a KMeans model using the loop counter for n_clusters
    model = KMeans(n_clusters=k, random_state=1)
    
    # 2. Fit the model to the data using df_scaled
    model.fit(df_scaled)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(model.inertia_)

In [129]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [131]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", width=800, height=400)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for `k` is 3.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [136]:
# Initialize the K-Means model using the best value for k
best_k = 3
model = KMeans(n_clusters=best_k, random_state=1)


In [138]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df_scaled)

KMeans(n_clusters=3, random_state=1)

In [140]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
clusters = model.predict(df_scaled)

# Print the resulting array of cluster values.
print(clusters)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 2 0 0 2 1 2 1 0
 0 1 0 0]


In [142]:
# Create a copy of the scaled DataFrame
df_scaled_clusters = df_scaled.copy()

In [144]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_scaled_clusters['Cluster'] = clusters

# Display the copy of the scaled DataFrame
df_scaled_clusters.head()

,price_change_percentage_24h,price_change_percentage_7d,Cluster
coin_id,,,
bitcoin,0.508529,0.493193,1
ethereum,0.185446,0.934445,1
tether,0.021774,-0.706337,0
ripple,-0.040764,-0.810928,0
bitcoin-cash,1.193036,2.000959,1


In [146]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_scaled_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters using Scaled Data"
)

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [150]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=2)

In [152]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_data = pca.fit_transform(df_scaled)

# View the scaled PCA data
pca_data[:5]

array([[-0.70832473, -0.0108444 ],
       [-0.79188247,  0.52962238],
       [ 0.48405906, -0.51485209],
       [ 0.60223751, -0.54458796],
       [-2.25849563,  0.57128782]])

In [154]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance is 1.

In [157]:
# Create a new DataFrame with the PCA data.
df_pca = pd.DataFrame(pca_data, columns=["PC1", "PC2"])

# Copy the crypto names from the original scaled DataFrame
df_pca["coin_id"] = df_scaled.index

# Set the coin_id column as index
df_pca.set_index("coin_id", inplace=True)

# Display the scaled PCA DataFrame
df_pca.head()

,PC1,PC2
coin_id,,
bitcoin,-0.708325,-0.010844
ethereum,-0.791882,0.529622
tether,0.484059,-0.514852
ripple,0.602238,-0.544588
bitcoin-cash,-2.258496,0.571288


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [161]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [163]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # Initialize the KMeans model using the current value of k
    model = KMeans(n_clusters=k, random_state=1)
    
    # Fit the model to the scaled PCA data
    model.fit(df_pca)
    
    # Append the model's inertia to the inertia list
    inertia_values.append(model.inertia_)

In [165]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [167]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Method for Optimal k",
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
    width=800,
    height=400
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value of `k` is 3.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it does not differ.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [171]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=best_k, random_state=1)

In [173]:
# Fit the K-Means model using the PCA data
model.fit(df_pca)

KMeans(n_clusters=3, random_state=1)

In [175]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
clusters = model.predict(df_pca)

# Print the resulting array of cluster values.
print(clusters)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 2 0 0 2 1 2 1 0
 0 1 0 0]


In [177]:
# Create a copy of the scaled PCA DataFrame
df_pca_clusters = df_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_pca_clusters['Cluster'] = clusters

# Display the copy of the scaled PCA DataFrame
df_pca_clusters.head()

,PC1,PC2,Cluster
coin_id,,,
bitcoin,-0.708325,-0.010844,1
ethereum,-0.791882,0.529622,1
tether,0.484059,-0.514852,0
ripple,0.602238,-0.544588,0
bitcoin-cash,-2.258496,0.571288,1


In [179]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_pca_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters using PCA"
)

:NdOverlay   [Cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [182]:
# Composite plot to contrast the Elbow curves
elbow_plot_original = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Method: Original Scaled Data vs PCA",
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
    color='blue',
    label="Original Scaled Data",
    width=800,
    height=400
)

# Plot the Elbow curve for the scaled PCA DataFrame
elbow_plot_pca = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    color='red',
    label="Scaled PCA Data",
    width=800,
    height=400
)

# Combine both elbow plots
elbow_plot_original + elbow_plot_pca

:Layout
   .Curve.Original_Scaled_Data :Curve   [k]   (inertia)
   .Curve.Scaled_PCA_Data      :Curve   [k]   (inertia)

In [184]:
# Composite plot to contrast the clusters
scatter_original = df_scaled_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c="predicted_clusters",
    cmap="viridis",
    title="Clusters: Original Scaled Data vs PCA",
    xlabel="24h Price Change (%)",
    ylabel="7d Price Change (%)",
    hover_cols=["coin_id"],
    width=800,
    height=400
)

# Scatter plot for clusters on the scaled PCA DataFrame
scatter_pca = df_pca_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    c="predicted_clusters",
    cmap="viridis",
    hover_cols=["coin_id"],
    width=800,
    height=400
)

# Combine both scatter plots
scatter_original + scatter_pca

:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .Scatter.II :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features in K-Means can make clustering faster and easier, but it may lose some important details. Choosing meaningful features ensures better clusters and removes noise.